In [1]:
import numpy as np
import skimage.io as io
import random
import os
import albumentations as A
import cv2
import pickle
import torch
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
from albumentations.pytorch import ToTensorV2
import glob
import time
from random import shuffle

In [2]:
torch.cuda.device_count()
device = torch.device('cuda:0')
torch.cuda.empty_cache()

In [3]:
mode = 'val'

load_folder = '/shared/data/COCOdataset2017/images/{}'.format(mode)
save_folder = '/shared/suhyeon/GCCdataset'
mask_folder = '{}/masks/{}'.format(save_folder, mode)
ann_file = '/shared/data/COCOdataset2017/annotations/instances_{}.json'.format(mode)
coco = COCO(ann_file)

batch_size = 1
input_size = (256, 256)
SMOOTH = 1e-6

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!


In [4]:
with open('./validAnns_val.pkl', 'rb') as f:
    validAnns = pickle.load(f)

with open('./top5_dict_val.pkl', 'rb') as f:
    top5_dict = pickle.load(f)
    
# def getImage(self, imageObj, img_folder, img_size):
#         # Read and normalize an image
#         train_img = cv2.imread(img_folder + '/' + imageObj['file_name'], cv2.IMREAD_COLOR)
#         train_img = cv2.cvtColor(train_img, cv2.COLOR_BGR2RGB)
#         train_img = cv2.resize(train_img, img_size) # (W,H)
#         return train_img


In [5]:
len(top5_dict.items())

6679

In [6]:
print(len(validAnns))

6679


In [7]:
# 1) load all images, masks
# torch XXXX opencv operation needed

images = {}
masks = {}

start = time.time()

# images folder glob -> transform (256,256)
for i, (catId, imgId, annId) in enumerate(validAnns):
    image = cv2.imread(load_folder + '/' + str(imgId).zfill(12) + '.jpg', cv2.IMREAD_COLOR)
    image = cv2.resize(image, input_size)
    images[imgId] = image
    # mask = trans(mask).type(torch.bool).to(device)
    
    mask = cv2.imread(mask_folder + '/' + str(annId) + '.png', cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, input_size)
    # mask = trans(mask).type(torch.bool).to(device)
    masks[annId] = mask
    
end = time.time()

print(f"data loaded: {end - start:.5f} sec")

data loaded: 51.37266 sec


In [8]:
print(len(images), len(masks))

3669 6679


In [9]:
# masks[6000].shape

In [10]:
# 2) not load images, just get images from file paths
### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

kernel = np.ones((3, 3), np.uint8)
SMOOTH = 1e-6
total = 0

In [11]:
from collections import defaultdict
print('# class:', len(coco.getCatIds()))
cnt_dict = defaultdict(int)
top5_list = list(top5_dict.items())
shuffle(top5_list)

# class: 80


In [12]:
# top5_list[0]

In [13]:
len(top5_list)

6679

In [17]:
for i, ((catId_1, imgId_1, annId_1), cands) in enumerate(top5_list):
    # start = time.time()
    # cat_id = coco.loadAnns(annId_1)[0]['category_id']
    # print(catId_1, cnt_dict[catId_1], annId_1, cands)

    img1 = images[imgId_1]
    mask1 = masks[annId_1]
    
    # fg1 = cv2.bitwise_and(img1, img1, mask = mask1) 
    invert1 = cv2.bitwise_not(mask1)//255
    bg1 = cv2.bitwise_and(img1, img1, mask = invert1)
    
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_bg' + '.png', bg1)
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_fg' + '.png', fg1)
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_gt' + '.png', img1)
    # cv2.imwrite(save_folder + '/val/src/' + str(annId_1).zfill(12) + '_mask' + '.png', mask1)
    
    if cnt_dict[catId_1] < 5:
        imgId_2, annId_2, _ = cands[np.random.randint(5)] #top5
        # for j, (imgId_2, annId_2, _) in enumerate(cands):
        if imgId_2 != 0 and annId_2 != 0:
            
            print(catId_1, imgId_1, annId_1, imgId_2, annId_2, cnt_dict[catId_1])

            img2 = images[imgId_2]
            mask2 = masks[annId_2]
            
            fg2 = cv2.bitwise_and(img2, img2, mask = mask2)
            
            path = save_folder + '/val_bg/' + str(annId_1).zfill(12) + '_' + str(annId_2).zfill(12) + '_bg' + '.png'
            
            if not os.path.isfile(path):
                cnt_dict[catId_1] += 1
                cv2.imwrite(save_folder + '/val_bg/' + str(annId_1).zfill(12) + '_' + str(annId_2).zfill(12) + '_bg' + '.png', bg1)
                cv2.imwrite(save_folder + '/val_bg/' + str(annId_1).zfill(12) + '_' + str(annId_2).zfill(12) + '_fg' + '.png', fg2)
                cv2.imwrite(save_folder + '/val_bg/' + str(annId_1).zfill(12) + '_' + str(annId_2).zfill(12) + '_mask' + '.png', mask2)

#             fg2 = cv2.bitwise_and(img2, img2, mask = mask2) 
#             invert2 = cv2.bitwise_not(mask2)//255
#             bg2 = cv2.bitwise_and(img2, img2, mask = invert2)

#             com = cv2.copyTo(fg2, mask2, img1.copy())

#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_bg' + '.png', bg2)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_com' + '.png', com)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_fg' + '.png', fg2)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_gt' + '.png', img2)
#             # cv2.imwrite(save_folder + '/val/tgt/' + str(annId_1).zfill(12) + '_tgt_' + str(annId_2).zfill(12) + '_mask' + '.png', mask2)
    
#     end = time.time()
#     elapsed = end - start
#     total += elapsed
    
    # print("[{}/{}]".format(i, len(top5_dict)), f"average time: {total/(i+SMOOTH):.5f} sec", end='\r')

80 232348 1128607 66841 1128597 1
36 193429 615389 304817 619320 2
34 52891 601915 221693 601999 1
90 160666 342306 521405 342858 0
40 407298 634391 27932 634731 1
89 350002 1676045 257169 1188308 2
89 257169 1188308 350002 1676045 2
34 221693 601999 6954 605276 1


In [15]:
print(cnt_dict)

defaultdict(<class 'int'>, {17: 5, 8: 5, 1: 5, 54: 5, 42: 5, 73: 5, 86: 5, 18: 5, 15: 5, 64: 5, 19: 5, 78: 5, 65: 5, 59: 5, 47: 5, 61: 5, 28: 5, 7: 5, 4: 5, 5: 5, 62: 5, 58: 5, 11: 5, 88: 5, 72: 5, 24: 5, 38: 5, 77: 5, 6: 5, 3: 5, 10: 5, 51: 5, 20: 5, 2: 5, 75: 5, 25: 5, 82: 5, 52: 5, 9: 5, 76: 5, 57: 5, 33: 5, 21: 5, 70: 5, 60: 5, 84: 5, 16: 5, 22: 5, 67: 5, 63: 5, 44: 5, 81: 5, 46: 5, 56: 5, 23: 5, 55: 5, 49: 5, 79: 5, 43: 5, 14: 5, 13: 5, 53: 5, 41: 5, 31: 5, 48: 5, 87: 5, 80: 1, 85: 5, 27: 5, 32: 5, 37: 0, 40: 1, 36: 2, 50: 5, 35: 0, 34: 1, 74: 1, 90: 0, 89: 2, 39: 0})


In [16]:
# print(cnt_dict)
    
#     f, ax = plt.subplots(nrows=1, ncols=5, figsize=(8, 8))
#     f.tight_layout()

#     ax[0].imshow(bg.squeeze())
#     ax[0].set_title("bg")
#     ax[0].axis('off')

#     ax[1].imshow(fg.squeeze())
#     ax[1].set_title("fg")
#     ax[1].axis('off')
    
#     ax[2].imshow(mask1.squeeze())
#     ax[2].set_title("mask1")
#     ax[2].axis('off')
    
#     ax[3].imshow(img1.squeeze())
#     ax[3].set_title("img1")
#     ax[3].axis('off')
    
#     ax[4].imshow(invert.squeeze())
#     ax[4].set_title("invert")
#     ax[4].axis('off')

# image save with title 'annId_imgtype_i' (including 0)
# warning: RGB BGR before save
# imgtype: [bg, fg, gt, mask]